Image Classification using Deep Learning
Author: Allmin Susaiyah

Name of student:

In this notebook, image classification using deep learning is covered. shift + Enter will excecute each cell

In [5]:
from keras.datasets import mnist

In [6]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os, shutil
from PIL import Image
from glob import glob
from tqdm import tqdm

In [7]:
if not os.path.exists('result_images'):
    os.makedirs('result_images')
if os.path.exists('input_images'):
    shutil.rmtree('input_images')
os.makedirs('input_images')

In [ ]:
#Acquiring data and assigning them into folders
Total_Images = 70000
if len(glob("input_images/*/*/*.png")) < Total_Images:
    print("aquiring dataset")
    (x_train, y_train), (x_val, y_val) = mnist.load_data()
    
    for ind,(im,y) in tqdm(enumerate(zip(x_train, y_train))):
        IM = Image.fromarray(im)
        op_dir = "input_images/train/{}/".format(y)
        if not os.path.exists(op_dir):
            os.makedirs(op_dir)
        IM.save(op_dir + "image_{:06d}.png".format(ind))


    for ind,(im,y) in tqdm(enumerate(zip(x_val, y_val))):
        IM = Image.fromarray(im)
        op_dir = "input_images/validation/{}/".format(y)
        if not os.path.exists(op_dir):
            os.makedirs(op_dir)
        IM.save(op_dir + "image_{:06d}.png".format(ind))

    

aquiring dataset


47292it [02:10, 318.47it/s]

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12
# input image dimensions
img_rows, img_cols = 28, 28
# loading images
list_all_images = glob("input_images/*/*/*.*")
x_train = []; y_train = []; x_val = []; y_val = []
for ind,image_path in tqdm(enumerate(list_all_images)):
    mode,label,file_name = image_path.split(os.path.sep)[-3:]
    label = int(label)
    
    im = np.array(Image.open(image_path))
    if mode == 'train':
        x_train.append(im)
        y_train.append(label)
    elif mode == 'validation':
        x_val.append(im)
        y_val.append(label)
        
x_train = np.array(x_train)
x_val = np.array(x_val)
# (x_train, y_train), (x_val, y_test) = mnist.load_data()


In [ ]:
print(len(y_train))
# [img_rows, img_cols] = x_train.shape[1:3]

In [ ]:
# import requests
# requests.packages.urllib3.disable_warnings()
# import ssl
# try:
#    _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#    # Legacy Python that doesn't verify HTTPS certificates by default
#    pass
# else:
#    # Handle target environment that doesn't support HTTPS verification
#    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
#preprocessing
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'test samples')


In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.summary()
# my_callbacks = [keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5')]

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_pred = model.predict(x_val)


In [ ]:
y_pred_int = np.argmax(y_pred,axis=1)
y_val_int = np.argmax(y_val,axis=1)

In [ ]:
model.save_weights("model.h5")

In [ ]:
#displaying test images with predicted label
for ind,(im,ya,yp) in enumerate(zip(x_val,y_val_int,y_pred_int)):
    plt.imshow(im[:,:,0],cmap='gray')
    plt.title("actual:{}, predicted:{}".format(ya,yp))
    plt.savefig("result_images/image_{:06d}_actual_{}_predicted_{}".format(ind,ya,yp))
    plt.show()
    if ind == 100:
        break
   
    